# Tutoriel d'usage de l'API

In [1]:
import urllib.parse

domain = "51.91.251.0"
port = 3000
host = f"http://{domain}:{port}"
path = lambda x: urllib.parse.urljoin(host, x)

## Creating an avatar

The admin of the competition can create users. A user can then create as many avatars as they wish.

In [2]:
import requests

user_id = '12e1e6fc-456b-4738-ba59-b90c3386f583'
name = 'ema-first-avatar'
r = requests.post(path(f'avatars/{user_id}/{name}'))
r

ConnectionError: HTTPConnectionPool(host='51.91.251.0', port=3000): Max retries exceeded with url: /avatars/12e1e6fc-456b-4738-ba59-b90c3386f583/ema-first-avatar (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f356d649550>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [3]:
name = 'max-second-avatar'
r = requests.post(path(f"avatars/{user_id}/{name}"))
r

<Response [200]>

In [4]:
r = requests.get(path(f"avatars/{user_id}"))
for avatar in r.json():
    print(avatar['id'], avatar['name'])

5 max-first-avatar
6 max-second-avatar


## Making pricing requests

In [5]:
params = {
    "avatar_name": "max-first-avatar",
    "language": "hungarian",
    "city": "copenhagen",
    "date": 2,
    "mobile": 0,
}
r1 = requests.get(path(f"pricing/{user_id}"), params=params)
r1.json()

{'prices': [{'hotel_id': 720, 'price': 467, 'stock': 2},
  {'hotel_id': 434, 'price': 114, 'stock': 0},
  {'hotel_id': 349, 'price': 317, 'stock': 1},
  {'hotel_id': 367, 'price': 96, 'stock': 0},
  {'hotel_id': 952, 'price': 94, 'stock': 0},
  {'hotel_id': 528, 'price': 432, 'stock': 2},
  {'hotel_id': 990, 'price': 168, 'stock': 0},
  {'hotel_id': 60, 'price': 94, 'stock': 0},
  {'hotel_id': 834, 'price': 91, 'stock': 0},
  {'hotel_id': 757, 'price': 151, 'stock': 1},
  {'hotel_id': 531, 'price': 204, 'stock': 2},
  {'hotel_id': 772, 'price': 327, 'stock': 0},
  {'hotel_id': 664, 'price': 472, 'stock': 2},
  {'hotel_id': 332, 'price': 333, 'stock': 1},
  {'hotel_id': 113, 'price': 89, 'stock': 0},
  {'hotel_id': 445, 'price': 89, 'stock': 0},
  {'hotel_id': 695, 'price': 202, 'stock': 2},
  {'hotel_id': 142, 'price': 390, 'stock': 4},
  {'hotel_id': 547, 'price': 202, 'stock': 2},
  {'hotel_id': 793, 'price': 167, 'stock': 0},
  {'hotel_id': 994, 'price': 193, 'stock': 2},
  {'hotel_

In [6]:
params = {
    "avatar_name": "max-first-avatar",
    "language": "hungarian",
    "city": "copenhagen",
    "date": 3,
    "mobile": 0,
}
r = requests.get(path(f"pricing/{user_id}"), params=params)
r.status_code, r.json()['detail']

(422, 'A pricing request for this avatar already exists for a sooner date')

In [7]:
params = {
    "avatar_name": "max-second-avatar",
    "language": "hungarian",
    "city": "amsterdam",
    "date": 1,
    "mobile": 1,
}
r2 = requests.get(path(f"pricing/{user_id}"), params=params)
r2.json()

{'prices': [{'hotel_id': 12, 'price': 76, 'stock': 0},
  {'hotel_id': 567, 'price': 89, 'stock': 0},
  {'hotel_id': 630, 'price': 150, 'stock': 0},
  {'hotel_id': 307, 'price': 123, 'stock': 0},
  {'hotel_id': 807, 'price': 396, 'stock': 1},
  {'hotel_id': 317, 'price': 182, 'stock': 1},
  {'hotel_id': 809, 'price': 396, 'stock': 1},
  {'hotel_id': 269, 'price': 148, 'stock': 0},
  {'hotel_id': 821, 'price': 107, 'stock': 0},
  {'hotel_id': 441, 'price': 190, 'stock': 1},
  {'hotel_id': 603, 'price': 349, 'stock': 1},
  {'hotel_id': 557, 'price': 109, 'stock': 0},
  {'hotel_id': 588, 'price': 169, 'stock': 0},
  {'hotel_id': 211, 'price': 144, 'stock': 0},
  {'hotel_id': 657, 'price': 250, 'stock': 1},
  {'hotel_id': 517, 'price': 168, 'stock': 0},
  {'hotel_id': 85, 'price': 153, 'stock': 0},
  {'hotel_id': 271, 'price': 79, 'stock': 0},
  {'hotel_id': 103, 'price': 176, 'stock': 7},
  {'hotel_id': 163, 'price': 283, 'stock': 1},
  {'hotel_id': 450, 'price': 153, 'stock': 16},
  {'hot

## Build a training set

In [8]:
import pandas as pd

pricing_requests = []

requests = [r1, r2]
for r in requests:
    pricing_requests.append(
        pd.DataFrame(r.json()['prices']).assign(**r.json()['request'])
    )

pricing_requests = pd.concat(pricing_requests)
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id
0,720,467,2,copenhagen,2,hungarian,0,5
1,434,114,0,copenhagen,2,hungarian,0,5
2,349,317,1,copenhagen,2,hungarian,0,5
3,367,96,0,copenhagen,2,hungarian,0,5
4,952,94,0,copenhagen,2,hungarian,0,5


In [9]:
len(pricing_requests)

255

In [10]:
hotels = pd.read_csv('../input/defi-ia-2023/features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,720,467,2,copenhagen,2,hungarian,0,5,Chillton Worldwide,Chill Garden Inn,0,0,2
1,434,114,0,copenhagen,2,hungarian,0,5,Accar Hotels,Ibas,0,0,0
2,349,317,1,copenhagen,2,hungarian,0,5,Yin Yang,Royal Lotus,0,1,0
3,367,96,0,copenhagen,2,hungarian,0,5,Yin Yang,8 Premium,0,0,0
4,952,94,0,copenhagen,2,hungarian,0,5,Independant,Independant,0,0,0


In [11]:
sample_submission = pd.read_csv('../input/defi-ia-2023/sample_submission.csv')
sample_submission.head()

,index,price
0,0,191.0
1,1,191.0
2,2,191.0
3,3,191.0
4,4,191.0


In [12]:
sample_submission.to_csv('submission.csv', index=False)